# Projection of 2D ROOT data into 1D slice
Comparison of 1D slices of ROOT and McStas data
## Import libraries

In [ ]:
import uproot
import os
import numpy as np
import scipp as sc
import plopp as pp
import ipywidgets
import dataconfig  # to get paths to data

%matplotlib widget

## Import 2D McStas data

In [ ]:
path_to_mcstas2D_file = os.path.join(dataconfig.data_mcstas, 'monitor_tx_DENEX.dat')

assert os.path.isfile(path_to_mcstas2D_file), \
    'There is an issue with the chosen McStas output datafile'

# Extract shape of output data
# these values will be used to determine how many lines to read one array
with open(path_to_mcstas2D_file, 'r') as file:
    for line in file:
        if "array_2d" in line:
            type_array = line.rstrip()
            start = type_array.find('(') + 1
            end = type_array.find(')', start)
            nx_value, ny_value = map(int, type_array[start:end].split(','))
        if "xylimits" in line:
            xmin, xmax,  ymin, ymax = np.array(line.split(':')[1].split()).astype(float)

print(f'Limits of x- and y-axis: {xmin}, {xmax}, {ymin}, {ymax}\nNumber of points: nx={nx_value}, ny={ny_value}')

In [ ]:
data2d = np.genfromtxt(path_to_mcstas2D_file, max_rows=ny_value)

#flip data along y axis 
# data2d_mcstas = np.flip(data2d, 0)

da_mcstas = sc.DataArray(data=sc.array(dims=['x', 'tof'], values=np.flip(data2d, 0)),
                          coords={'tof': sc.linspace('tof', xmin, xmax, num=nx_value, unit='s'),
                                  'x': sc.linspace('x', ymin, ymax, num=ny_value, unit='m')})

In [ ]:
pp.plot(da_mcstas, title='monitor_tx_DENEX.dat')

## Select spatial range of McStas data

In [ ]:
ydim = 'x'

def add_mask(da, trunc_range):
    min_tr, max_tr = trunc_range  
    
    out = da.copy(deep=False)
    if ydim in out.masks:
        del out.masks[ydim]
    out.masks[ydim] = ((out.coords[ydim] > out.coords[ydim][max_tr]) | (out.coords[ydim] < out.coords[ydim][min_tr]))
    return out

in_node = pp.input_node(da_mcstas)
in_node.name = 'Input node'

# slider
max_slider = da_mcstas.sizes[ydim]
slider = ipywidgets.IntRangeSlider(value=[0, max_slider-1],
                                   min=0, 
                                   max=max_slider-1,
                                   description='x indices')

slider_node = pp.widget_node(slider)
slider_node.name = 'Slider node'

# add mask
add_mask_node = pp.node(add_mask)(in_node, trunc_range=slider_node)
add_mask_node.name = 'Show masks'

fig = pp.figure2d(add_mask_node, title='McStas')

sum_node = pp.node(sc.sum, dim=ydim)(add_mask_node)
sum_node.name = 'Sum'

fig1d = pp.figure1d(sum_node, grid=True, linestyle='solid', marker='')
ipywidgets.VBox([slider, fig, fig1d])

## Import ROOT data

In [ ]:
assert os.path.exists(dataconfig.data_root), 'The path does not exist.'

ROOTfile = 'Spectrum03_DENEX006_1_18-02-05_0000.root'

dir_with_data = 'Meas_3'

data_to_plot = 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'

file_to_open = os.path.join(dataconfig.data_root, ROOTfile)

assert os.path.isfile(file_to_open), 'There is an issue with the file to be opened.'

# Some metadata related to TOF channel for ROOT file
tof_tick = 25e-6 # in milliseconds (25 ns)

In [ ]:
with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key in myFile.keys():
        if 'BoardParam_run' in str(key):
            myObject = myFile[key]
            nb_xbins = int(myObject.member('fEntries'))
                        
            for i in range(nb_xbins):
                if 'TOF_Time_Channel_Width' in myObject.axis(axis=0).labels()[i]:
                    TOF_Time_Channel_Width = myObject.counts(False)[i]
                elif 'TOF_Window_Delay_Register' in myObject.axis(axis=0).labels()[i]:
                    TOF_Window_Delay_Register = myObject.counts(False)[i]
                
        # 2D contourplot
        if data_to_plot in str(key):  
            # x_max, x_min, y_max, y_min defined from .root file
            xaxis_root = myFile[key].axis(axis=0).edges()[:-1]
            yaxis_root = myFile[key].axis(axis=1).edges()[:-1]
            
            x_min_root = min(xaxis_root)  
            x_max_root = max(xaxis_root)  
            y_min_root = min(yaxis_root) 
            y_max_root = max(yaxis_root)  
        
            # fill 2d matrice with inverted y-axis
            data2d_root =  np.flip(myFile[key].counts(False), 1)

In [ ]:
da_root = sc.DataArray(data=sc.array(dims=['x', 'tof'], values=data2d_root.transpose().astype('float64')),
                      coords={
                      'tof': sc.array(dims=['tof'], values=xaxis_root),
                      'x': sc.array(dims=['x'], values=yaxis_root)})
da_root

In [ ]:
pp.plot(da_root, title='ROOT')

In [ ]:
ydim = 'x'

def add_mask(da, trunc_range):
    min_tr, max_tr = trunc_range  
    
    out = da.copy(deep=False)
    if ydim in out.masks:
        del out.masks[ydim]
    out.masks[ydim] = ((out.coords[ydim] > out.coords[ydim][max_tr]) | (out.coords[ydim] < out.coords[ydim][min_tr]))
    return out

in_node = pp.input_node(da_root)
in_node.name = 'Input node'

# slider
max_slider = da_root.sizes[ydim]
slider = ipywidgets.IntRangeSlider(value=[0, max_slider-1],
                                   min=0, 
                                   max=max_slider-1,
                                   description='x indices')

slider_node = pp.widget_node(slider)
slider_node.name = 'Slider node'

# add mask
add_mask_node = pp.node(add_mask)(in_node, trunc_range=slider_node)
add_mask_node.name = 'Show masks'

fig = pp.figure2d(add_mask_node)

sum_node = pp.node(sc.sum, dim=ydim)(add_mask_node)
sum_node.name = 'Sum'

fig1d = pp.figure1d(sum_node, grid=True, linestyle='solid', marker='')
ipywidgets.VBox([slider, fig, fig1d])

## Creating and comparing 1D projections of ROOT and McStas data

### Rescale TOF for ROOT data
Use metadata of ROOT file to add units and offset to the x-axis

In [ ]:
xaxis_root_rescaled = (xaxis_root * TOF_Time_Channel_Width + TOF_Window_Delay_Register) * tof_tick 

da_root_rescaled = da_root.copy()
da_root_rescaled.coords['tof'] = sc.array(
    dims=['tof'], 
    values=xaxis_root_rescaled/1000, 
    unit=sc.Unit('s')
)
da_root_rescaled

### Rescale counts for ROOT

In [ ]:
da_root_rescaled /= da_root.max()
da_root_rescaled

### Rescale counts for McStas 

In [ ]:
da_mcstas_rescaled = da_mcstas.copy()
da_mcstas_rescaled /= da_mcstas.max()

## Compare ROOT and McStas 1D profile

In [ ]:
in_nodem = pp.input_node(da_mcstas_rescaled)
in_nodem.name = 'Input node mcstas'

in_noder = pp.input_node(da_root_rescaled)
in_noder.name = 'Input node root'

# sliders
max_sliderm = da_mcstas_rescaled.sizes[ydim]
slider_mcstas = ipywidgets.IntRangeSlider(value=[0, max_sliderm-1],
                                   min=0, 
                                   max=max_sliderm-1,
                                   description='McStas index boundaries')

max_sliderr = da_root_rescaled.sizes[ydim]
slider_root = ipywidgets.IntRangeSlider(value=[0, max_sliderr-1],
                                   min=0, 
                                   max=max_sliderr-1,
                                   description='ROOT index boundaries')

slider_nodem = pp.widget_node(slider_mcstas)
slider_nodem.name = 'Slider node mcstas'

slider_noder = pp.widget_node(slider_root)
slider_noder.name = 'Slider node root'

# add mask
add_mask_nodem = pp.node(add_mask)(in_nodem, trunc_range=slider_nodem)
add_mask_nodem.name = 'Show masks mcstas'

add_mask_noder = pp.node(add_mask)(in_noder, trunc_range=slider_noder)
add_mask_noder.name = 'Show masks root'

fig_mcstas = pp.figure2d(add_mask_nodem)

fig_root = pp.figure2d(add_mask_noder)

# sum
sum_nodem = pp.node(sc.sum, dim=ydim)(add_mask_nodem)
sum_nodem.name = 'Sum mcstas'

sum_noder = pp.node(sc.sum, dim=ydim)(add_mask_noder)
sum_noder.name = 'Sum root'

# # fig1d = pp.figure1d({'root': sum_noder, 'mcstas': sum_nodem}, linestyle='solid', marker='')
fig1d = pp.figure1d(sum_noder, sum_nodem, linestyle='solid', marker='', grid=True)

slider_fig_mcstas = ipywidgets.VBox([slider_mcstas, fig_mcstas])
slider_fig_root = ipywidgets.VBox([slider_root, fig_root])

ipywidgets.VBox([ipywidgets.HBox([slider_fig_mcstas, slider_fig_root]), fig1d])